In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import json

In [ ]:
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36")

# Especifica la ubicación del chromedriver.exe
chrome_driver_path = './chromedriver.exe'

# Inicia el servicio del controlador de Chrome
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service , options=opts)

driver.get("https://listado.mercadolibre.com.py/laptops-accesorios-notebooks/")

In [ ]:
lista_detalles_de_productos = []
total_productos = driver.find_element(By.XPATH, '//span[@class="ui-search-search-result__quantity-results"]').text
#Extraer solo los dígitos de mi total de resultados de busqueda
total_productos_numero = int(''.join(filter(str.isdigit, total_productos)))
print("Número total de productos:", total_productos_numero)

producto_numero=0
while True:
    links_productos = driver.find_elements(By.XPATH, '//a[@class="ui-search-item__group__element ui-search-link__title-card ui-search-link"]')
    links_de_la_pagina = []
    for i in links_productos:
        links_de_la_pagina.append(i.get_attribute('href'))

    for link in links_de_la_pagina:
        try:
            driver.get(link)
            titulo = driver.find_element(By.XPATH, '//h1[@class="ui-pdp-title"]').text
            precio = driver.find_element(By.XPATH, '//span[@class="andes-money-amount__fraction"]').text
            
            descripcion_element = driver.find_element(By.XPATH, '//p[@class="ui-pdp-description__content"]')
            descripcion = ' '.join(descripcion_element.text.split())
            lista_detalles_de_productos.append({
                'link': link,
                'titulo': titulo,
                'precio': precio,
                'descripcion':descripcion,
            })

            #contabilizamos el avance de los productos
            producto_numero += 1
            print(f'Agregado producto numero: {producto_numero} de {total_productos_numero}')
            driver.back()
        except Exception as e:
            print(e)
            driver.back()

    try:
        # Hacer scroll hacia abajo hasta el final de la página
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Agregar una espera después del scroll
        wait = WebDriverWait(driver, 10)
        boton_siguiente = wait.until(EC.presence_of_element_located((By.XPATH, '//a[@class="andes-pagination__link ui-search-link"]/span[text()="Siguiente"]' )))
        boton_siguiente.click()
        
    except Exception as e:
        break
        print(e)

In [ ]:
print (len(lista_detalles_de_productos))
# Guarda la lista de productos totales en un archivo JSON
with open('productos_totales.json', 'w', encoding='utf-8') as json_file:
    json.dump(lista_detalles_de_productos, json_file, ensure_ascii=False, indent=4)